In [ ]:
import os
from difflib import SequenceMatcher
import csv

from src.plate_character_detector import PlateCharacterDetector
from src.plate_character_recognizer import PlateCharacterRecognizer

In [ ]:
# Instantiate object from Monopol core classes

plate_character_detector = PlateCharacterDetector()
plate_character_recognizer = PlateCharacterRecognizer()

In [ ]:
# Load plate character recognizer DCNN model, weights, and label classes

plate_character_recognizer.load_model()
plate_character_recognizer.load_weights()
plate_character_recognizer.load_classes_label()

In [ ]:
# TODO: read all dataset

PATH = "./data/testing"

datasets = []

for filename in os.listdir(PATH):
  image_path = PATH + "/{}".format(filename)
  plate_number = filename[:-4]
  datasets.append((image_path, plate_number))

print(datasets)

In [ ]:
def get_extracted_plate_number(image_path):
    plate_character_detector.load_image(file_path=image_path)
    (character_rois, crop_characters) = plate_character_detector.detect_characters()

    characters = ""

    for character in crop_characters:
        predicted_character, confidence_rate = plate_character_recognizer.predict(
            character)
        characters += predicted_character

    return characters

In [ ]:
get_extracted_plate_number(datasets[0][0])

In [ ]:
for image_path, plate_number in datasets:
  extracted_plate_number = get_extracted_plate_number(image_path)
  similarity_ratio = SequenceMatcher(None, plate_number, extracted_plate_number).ratio()
  print("Actual Plate Number: {}; Extracted Plate Number: {}; Similarity Ratio: {}".format(plate_number, extracted_plate_number, similarity_ratio))

In [ ]:
header = ['Actual Plate Number', 'Extracted Plate Number', 'Similarity Ratio']

with open('result.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    for image_path, plate_number in datasets:
      extracted_plate_number = get_extracted_plate_number(image_path)
      similarity_ratio = SequenceMatcher(None, plate_number, extracted_plate_number).ratio()
      data = [plate_number, extracted_plate_number, similarity_ratio]
      writer.writerow(data)